## Purpose of K-Means Clustering

The purpose of the notebook is to use annotatoins of each emoji to intelligently cluster emojis
* use 'emoji_webscraped_expanded.json' file to create categories

** Import Packages **

In [1]:
# import packages work bitch please meow
# meow meow
# meow
import nltk, re
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.data import find
import pandas as pd
from collections import defaultdict
from nltk.corpus import stopwords
import json

** Function to convert the text file into a list of 1) titles 2) descriptions 3) annotations **

In [2]:
df = pd.read_json("data/emoji_webscraped_expanded.json")

**Subset Dataframe to only annotations which either contain face or person**

In [3]:
def subset_annotations(_df):
    list_titles = [list(item) for item in list(_df)]
    index_face_person = [index for index,value in enumerate(list_titles) if 'face' in value] # or 'person' in value]
    # print(len(index_face_person))
    df_face_person = df.iloc[index_face_person]
    # print(df_face_person.shape)
    # df_face_person.head()
    return df_face_person

** return list of all annotations and the fifty top annotations **

In [4]:
def word_list(annotations, _common=100):
    stop_words = stopwords.words('english')
    total_list = [word for item in list(annotations) for word in item if word not in stop_words]
    top_list = nltk.FreqDist(total_list).most_common(_common)
    return total_list, top_list

** return list of fifty top words without frequency rate **

In [5]:
def top_justwords(top_list):
    top_words = [item[0] for item in top_list]
    return top_words

In [6]:
def face_by_topwords(top, df):
    top_justwords_array = [(top) for i in range(df.shape[0])]
    df['top_words'] = pd.Series(top_justwords_array, index=df.index)
    
    df['top_binary'] = [list(pd.Series(item).isin(list(df.annotations)[index])) for index, item in enumerate(df.top_words)]
    df = df.reset_index(drop=True)
    
    return df

In [7]:
from sklearn.cluster import KMeans

def Clustering(df_combined, n_clusters=5):
    _X = np.array(df_combined.ix[:,7:(df_combined.shape)[1]])
    k_means = KMeans(init='k-means++', n_clusters= n_clusters, n_init=20)
    k_means.fit(_X)
    k_means_labels = k_means.labels_
    k_means_cluster_centers = k_means.cluster_centers_
    k_means_labels_unique = np.unique(k_means_labels)
    ft = (k_means_labels, k_means_cluster_centers, k_means_labels_unique)
    labels = np.array(k_means_labels_unique)
    location = np.array(k_means_cluster_centers)
    labels_location = list(zip(labels, location))
    # person_df['cluster_label'] = pd.DataFrame(k_means_labels)
    print ("labels:\n %s, \n cluster centers:\n %s,\n  unique labels:\n %s" % ft)
    print(labels_location)
    return k_means_labels

In [8]:
def create_dictionary(df, labels):
    df_combined['k_means'] = list(k_means_labels)
    unique_labels = list(k_means_labels)
    
    # subset df to two fields needed
    subset = df_combined[['k_means','byteCode1']]
    subset = subset.values.tolist()
    
    # create dictionary from subset df
    dict_grouping = defaultdict(list)
    for key, date in subset:
        dict_grouping[key].append(date)
    
    return dict_grouping

## Call Functions

In [9]:
df_face_person = subset_annotations(df.annotations)
total_list, top_list = word_list(df_face_person.annotations, 50)
top_justwords = top_justwords(top_list)

df_features = pd.DataFrame(columns=top_justwords)
df_face_person = face_by_topwords(top_justwords, df_face_person)

data_features = list(df_face_person.top_binary)

df_features = pd.DataFrame(data_features, columns=top_justwords)
df_features = df_features.astype(int)

df_combined = pd.concat([df_face_person, df_features], axis=1, join_axes=[df_face_person.index])
k_means_labels = Clustering(df_combined, n_clusters=5)
dict_grouping = create_dictionary(df_combined, k_means_labels)

/Users/carlo_liquido/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/carlo_liquido/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


labels:
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 3 3 1 3 3 3 3 3 3 1 3 3 3 1 3 3 3
 1 3 3 1 3 3 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 1 2 2 1 3 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 1 0 0 0 1 3 3 1 3 3 1
 3 3 3 3 3 3 4 4 1 4], 
 cluster centers:
 [[  1.00000000e+00   1.00000000e+00   2.00000000e-01   5.55111512e-17
    2.00000000e-01   0.00000000e+00   1.00000000e+00   1.38777878e-17
    1.38777878e-17   9.00000000e-01   6.00000000e-01   0.00000000e+00
    2.00000000e-01  -1.38777878e-17   0.00000000e+00   5.00000000e-01
    6.93889390e-18   6.93889390e-18   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e-01
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   2.00000000e-01
    0.00000000e+00   0.00000000e+00   2.00000000e-01   2.00000000e-01
    0.00000000e+00   0.

## Output Emojis by Category

In [10]:
for key, value in dict_grouping.items():
    print(key)
    for _emoji in value:
        print(bytes("{0}".format(_emoji), 'ascii').decode('unicode-escape'), end="")
    print("\n")

0
👼😇😈👿👹👺💀👻👽👾

1
😀😁😎⛑😍😘🗣😗😙😚☺👀🙂🤗😂🤔😐😑😶🙄😏😣😥😮🤐😃😯😪😫😴😌🤓😛😜😝☹😄🙁😒😓😔😕😖🙃😷🤒🤕😅🤑😲😞😟😤😢😭😦😧😨😆😩😬😰😱😳😵😡😠😉☠🤖😊🗿😋

2
🌚🌛🌜🌝🌞

3
🐵🐶🐺🐱🦁🐯🐴🦄🐮🐷🐽🐭🐹🐰🐻🐼🐸🐲🐳🌬💩😺😸😹😻😼😽🙀😿😾

4
🙈🙉🙊

